In [112]:
import os
import pathlib
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras import backend as K

### Init model, load saved weights and read class names

In [131]:
class_names = np.array(sorted([item.name for item in data_dir.glob('*') if item.is_dir()]))
print(class_names)

model_input_shape = (224,224,3)

def loadCNNModel(saved_weights_path):
    K.clear_session()

    # Create model -> UPDATE if architecture changes
    model = Sequential([
        Conv2D(32, (5,5), activation='relu', input_shape=model_input_shape), # filters, kernel_size
        Conv2D(32, (5,5), activation='relu'),
        MaxPool2D(), #pool_size=2, padding="valid"
        Conv2D(32, (5,5), activation='relu'),
        Conv2D(32, (5,5), activation='relu'),
        MaxPool2D(), #2
        Flatten(),
        Dense(7, activation='softmax') # 7 is number of classes
    ])

    # Compile the model
    model.compile(loss="categorical_crossentropy",
                   optimizer=tf.keras.optimizers.Adam(),
                   metrics=["accuracy"])
    
    # Load the model from the saved Keras file
    model.load_weights(saved_weights_path)
    
    # Keras builds the GPU function the first time you call predict(). 
    # That way, if you never call predict, you save some time and resources. 
    # However, the first time you call predict is slightly slower than every other time.
    model.predict(np.array([np.ones(model_input_shape)]));
    
    return model;

['Am' 'Background' 'C' 'Dm' 'Em' 'F' 'G']


## Using Camera as model's input

In [125]:
import cv2
import threading
import ipywidgets as widgets
import matplotlib.pyplot as plt

from IPython.display import display, Image

!ls -ltrh /dev/video* # linux command to list available cameras

crw-rw---- 1 root video 81, 0 Nov 14 20:55 /dev/video0


#### Create the camera object
Remember to change device if incorrect for your system (based on out put of camera listing /dev/video\<camera_no>)

In [133]:
# Create a VideoCapture object to access the camera
# If "0" doesn't work for, try uncommenting the long init
def initCameraObject():
    # return cv2.VideoCapture("v4l2src device=/dev/video0 ! video/x-raw,format=YUY2,width=640,height=480,framerate=30/1 ! nvvidconv ! video/x-raw(memory:NVMM) ! nvvidconv ! video/x-raw, format=BGRx ! videoconvert ! video/x-raw, format=BGR ! appsink drop=1 ", cv2.CAP_GSTREAMER)
    return cv2.VideoCapture(0)

# Predict 
def predict(model, image):
    if(image.shape[0:2] != model_input_shape[0:2]): # check size of input image
        image = cv2.resize(image, model_input_shape[0:2], interpolation = cv2.INTER_AREA) 
        
    with session.as_default():
            with graph.as_default():
                predictions = model.predict( np.array([image]));
                
    pred_index = np.argmax(predictions);
    return class_names[pred_index]

### Video update callback

Define callback for camera. Every step in loop we fetch the image, rescale it to shape required by our model (if needed) and run a prediction. The prediction is diplayed in the bottom left corner of the video.

In [135]:
# Stop button widget
stopButton = widgets.ToggleButton(
    value=False,
    description='Stop',
    disabled=False,
    button_style='danger',
    tooltip='Description',
    icon='square'
)

# Display function
def view(model, button):
    cap = initCameraObject(); # init camera
    display_handle = display(None, display_id=True) # display inside Jupyter Notebook
    
    while True: # video "callback"
        ret, frame = cap.read() # read the frame
        
        if not ret or stopButton.value == True: # if frame can't be read or user clicked the "Stop" button
            cap.release()
            display_handle.update(None)
        
        prediction = predict(model, frame)
        
        # Display predicted class
        cv2.putText(frame, prediction, (10, frame.shape[0] - 10), font, 3, (0, 255, 0), 2, cv2.LINE_AA, False)
        
        _, frame = cv2.imencode('.jpeg', frame)
        display_handle.update(Image(data=frame.tobytes()))
           
            
# Run
# ================
display(stopButton) # show stop button

cnn_model = loadCNNModel('Models/model_1_weights.h5')

session = K.get_session()
graph = tf.get_default_graph()
graph.finalize() # finalize

thread = threading.Thread(target=view, args=(cnn_model, stopButton,))
thread.start()

ToggleButton(value=False, button_style='danger', description='Stop', icon='square', tooltip='Description')

None